# Set CPU/Memory resource requirements for SolrCloud

In [ ]:
import yaml
import itertools
from ckan_cloud_operator import kubectl

old_pods = [pod for pod in kubectl.get('pod')['items'] if pod['metadata']['labels'].get('app') == 'provider-solr-solrcloud-sc']
assert len(old_pods) == 3
old_containers = [container for old_pod in old_pods for container in old_pod['spec']['containers']]
assert len(old_containers) == 3

In [ ]:
import yaml
from IPython.core.display import HTML

HTML('<h3>OLD RESOURECS</h3>' + yaml.dump([old_container['resources'] for old_container in old_containers], default_flow_style=False).replace(' ', '&nbsp;').replace('\n','<br/>'))


## Set new resources

In [ ]:
import json
from ckan_cloud_operator.config import manager as config_manager

config_manager.set(
    key='container-spec-overrides', 
    value=json.dumps({
        "resources": {
            "requests": {
                "cpu": "2", 
                "memory": "4Gi"
            },
            "limits": {
                "cpu": "3",
                "memory": "6Gi"
            }
        }
    }),
    configmap_name='ckan-cloud-provider-solr-solrcloud-sc-config'
)

## Apply deployment: Dry Run

In [ ]:
from ckan_cloud_operator.providers.solr.solrcloud import manager as solrcloud_manager

solrcloud_manager.initialize(dry_run=True)

## Apply Deployment

For solrcloud, deployment must be handled manually, from an interactive terminal

You can open a terminal in Jupyter Lab or elsewhere and run:

Dry Run:

```
ckan-cloud-operator solr initialize --interactive --dry-run
```

Deploy interactively, only update the solrcloud pods:

```
ckan-cloud-operator solr initialize --interactive
```

While deployment is in progress, run the next step to check progress

## Check deployment progress

In [ ]:
import yaml
from ckan_cloud_operator import kubectl

pods = [pod for pod in kubectl.get('pod')['items'] if pod['metadata']['labels'].get('app') == 'provider-solr-solrcloud-sc']
pod_names = [pod['metadata']['name'] for pod in pods]
print(yaml.dump(pod_names, default_flow_style=False))

[print(kubectl.check_output(f'describe pod {pod_name}').decode()) for pod_name in pod_names]